# Design Patterns

## Let's start with an example

**Problem:** We have multiple components that want to get updated information from a single source

![alt](polling.svg)

![alt](push.svg)

In [1]:
class Publisher:  # The source
    def __init__(self):
        self.subscribers = []
    def add_subscriber(self, subscriber):
        self.subscribers.append(subscriber)
    def publish(self, data):
        for subscriber in self.subscribers:
            subscriber.notify(data)
        
class Subscriber:  # The components that get updates from the source
    def notify(self, data):
        pass

### Observer Pattern 
*(a.k.a Event-Subscriber, Publisher-Subscriber Listener)*

## Resources

* https://pyvideo.org/pyohio-2012/python-design-patterns-1.html
* https://python-patterns.guide/
* https://refactoring.guru
* https://diagrams.mingrammer.com/